# 네이버 영화 리뷰 긍/부정 분류

목표 : 네이버 영화 리뷰 글을 딥러닝 모델에 적용하여 긍/부정 분류를 높은 성능으로 진행해보자

데이터 : 

In [1]:
import pandas as pd
import urllib.request
%matplotlib inline
import matplotlib.pyplot as plt
import re
from konlpy.tag import Okt
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from collections import Counter
import os

In [2]:
# 데이터를 읽어봅시다. 
train_data_path = os.getenv('HOME') + '/aiffel/exp/exp4/dataset/sentiment_classification/ratings_train.txt'
test_data_path = os.getenv('HOME') + '/aiffel/exp/exp4/dataset/sentiment_classification/ratings_test.txt'

# train / test 각각 불러와서
train_data = pd.read_table(train_data_path)
test_data = pd.read_table(test_data_path)

# train data head 만 보자
train_data.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [12]:
# test data head
test_data.head()

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [3]:
from konlpy.tag import Mecab
# mecab tokenizer 를 사용한다.
tokenizer = Mecab()
# 아래 stopword 들은 형태소로써 제외한다. 
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

def load_data(train_data, test_data, num_words=10000):
    # document 부분만 필요하여 꺼내쓰겠다.
    train_data.drop_duplicates(subset=['document'], inplace=True)
    # 빈 거를 제외한다. NAN 제거
    train_data = train_data.dropna(how = 'any') 
    # document 부분만 필요하여 꺼내쓰겠다.
    test_data.drop_duplicates(subset=['document'], inplace=True)
    # 빈 거를 제외한다. NAN 제거
    test_data = test_data.dropna(how = 'any') 

    # return 할 x_train list 를 초기화 해준 뒤에
    X_train = []
    # document 안의 영화평을 긁어서 각각
    for sentence in train_data['document']:
        # 형태소 단위로 쪼갠 뒤에
        temp_X = tokenizer.morphs(sentence) # 토큰화
        # 불용어 제거한다.
        temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
        # 담는다
        X_train.append(temp_X)

    # train 과 동일하다
    X_test = []
    for sentence in test_data['document']:
        temp_X = tokenizer.morphs(sentence) # 토큰화
        temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
        X_test.append(temp_X)

    # list 를 np concatenate
    words = np.concatenate(X_train).tolist()
    # 단어 숫자를 세어서 'the':100 개 이런 식으로 dict 형태로 표현한다.
    counter = Counter(words)
    counter = counter.most_common(10000-4)
    vocab = ['<PAD>', '<BOS>', '<UNK>', '<UNUSED>'] + [key for key, _ in counter]
    word_to_index = {word:index for index, word in enumerate(vocab)}

    def wordlist_to_indexlist(wordlist):
        return [word_to_index[word] if word in word_to_index else word_to_index['<UNK>'] for word in wordlist]

    X_train = list(map(wordlist_to_indexlist, X_train))
    X_test = list(map(wordlist_to_indexlist, X_test))

    return X_train, np.array(list(train_data['label'])), X_test, np.array(list(test_data['label'])), word_to_index

X_train, y_train, X_test, y_test, word_to_index = load_data(train_data, test_data) 

In [4]:
X_train[1]

[1028, 4, 11, 519, 549, 687, 5, 120, 4, 70, 1639, 52, 925, 1006, 15, 44, 391]

In [5]:
y_train[1]

1

In [6]:
X_test[1]

[2]

In [7]:
y_test[1]

0

In [11]:
len(word_to_index)    # 10000
word_to_index['ㅋ']   # 130

130